<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/loading_worksheet_study_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [ ]:
!pip install ipython ipykernel --upgrade

# Set Up Environment

### Import required libraries

In [ ]:
!pip install -q canopy

In [ ]:
import canopy
import logging
import matplotlib.pyplot as plt
import nest_asyncio
import re
from typing import List, Dict, NamedTuple, Optional
import statistics
import pandas as pd
import numpy as np

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

### Authenticate

In [ ]:
authentication_data = canopy.prompt_for_authentication()
session = canopy.Session(authentication_data)

# Setup the Example

Ask which worksheet to query.

In [ ]:
worksheet_name_or_id = input("Worksheet name or ID: ")

Load the worksheet.

In [ ]:
worksheet = None
if re.match('^[0-9a-f]{32}$', worksheet_name_or_id):
    logging.info(f'Worksheet ID provided: {worksheet_name_or_id}')
    worksheet = await canopy.load_config(session, worksheet_name_or_id)
else:
    logging.info(f'Worksheet name provided: {worksheet_name_or_id}')
    worksheet = await canopy.find_config(session, 'worksheet', worksheet_name_or_id)

worksheet

This function does all the work of collating the data we've requested.

In [ ]:
sim_types_to_load = ['QuasiStaticLap', 'DynamicLap']

class NameAndUnits(NamedTuple):
    name: str
    units: str

async def load_worksheet_job_data(
    worksheet: canopy.ConfigResult, 
    scalar_channels: Optional[List[NameAndUnits]],
    vector_channels: Optional[List[NameAndUnits]],
    sim_types_to_load: List[str]) -> pd.DataFrame:

    if scalar_channels is None:
        scalar_channels = []

    if vector_channels is None:
        vector_channels = []

    scalar_results_requested = len(scalar_channels) > 0
    vector_results_requested = len(vector_channels) > 0

    jobs_data: List[Dict] = []

    for row_index, row in enumerate(worksheet.data.rows):
        if 'reference' in row.study:
            study_id = row.study.reference.targetId
            logging.info(f'Loading metadata for study ID {study_id}')
            
            study_metadata = await canopy.load_study(
                session,
                study_id,
                include_study_sim_types=True)
            
            logging.info('Found sim types: %s', ','.join(study_metadata.sim_types))
            filtered_sim_types = [sim_type for sim_type in study_metadata.sim_types if sim_type in sim_types_to_load]
            
            is_exploration = study_metadata.simulation_count > 1
            logging.info(f'is_exploration: {is_exploration}')

            for sim_type in filtered_sim_types:
                logging.info(f'Loading data for {sim_type}')
                study = await canopy.load_study(
                    session,
                    study_id,
                    sim_type,
                    channel_names=[v.name for v in vector_channels],
                    
                    # If the study was an exploration it is more efficient to load
                    # scalar results from the aggregated study scalar results file.
                    include_study_scalar_results = scalar_results_requested and is_exploration,
                    
                    # If the study was a single job then the aggregated file doesn't exist,
                    # so we will load the scalar results for the individual job.
                    include_job_scalar_results = scalar_results_requested and not is_exploration)

                converted_scalars: Dict[List[float]] = {}
                if scalar_results_requested:
                    for item in scalar_channels:
                        if is_exploration:
                            # When we load from the aggregated file we need to fully qualify the
                            # scalar result channel name with the sim type, because the aggregated
                            # file contains the results for all sim types which were run in the job.
                            scalar_channel_name = f'{item.name}:{sim_type}'
                            converted_scalars[item.name] = study.scalar_as(scalar_channel_name, item.units)
                        else:
                            # When we load from the individual job scalar results we don't need to fully
                            # qualify the channel name, because it has only loaded the scalar results for the
                            # sim type we requested in the load_study method.
                            scalar_channel_name = item.name
                            converted_scalars[item.name] = [study.jobs[0].scalar_as(scalar_channel_name, item.units)]

                study_scalars_tLapTotal: List[number]
                for job_index in range(0, study_metadata.simulation_count):
                    
                    # The list of jobs is only populated if required by the data we have requested.
                    # If we don't request any vector data, and the scalar results were loaded from the
                    # aggregated scalar results file, then it will not be populated because 
                    # loading data for individual jobs was not required.
                    job = study.jobs[job_index] if study.jobs else None

                    job_data: Dict = {
                        'row_index': row_index,
                        'study_id': study_id,
                        'study_name': study.document.name,
                        'sim_type': sim_type,
                        'job_index': job_index,
                        'job_name': job.document.name if job else '',
                    }

                    for item in vector_channels:
                        data = job.vector_as(item.name, item.units)
                        job_data[f'{item.name}_Min'] = min(data) if data is not None else np.nan
                        job_data[f'{item.name}_Mean'] = statistics.mean(data) if data is not None else np.nan
                        job_data[f'{item.name}_Max'] = max(data) if data is not None else np.nan

                    for item in scalar_channels:
                        scalar_data = converted_scalars[item.name]
                        job_data[item.name] = scalar_data[job_index] if scalar_data is not None else np.nan

                    jobs_data.append(job_data)
    
    return pd.DataFrame(jobs_data)

# Example: Loading Worksheet Study Data

In [ ]:
# This example doesn't request any vector data.
# It will complete very quickly because all the scalar results
# can be loaded from the single aggregated scalar results file for a study.
await load_worksheet_job_data(
    worksheet,
    [
        NameAndUnits('tLapTotal', 's'),
        NameAndUnits('mFuelTotal', 'kg'),
    ],
    None,
    [
        'QuasiStaticLap',
        'DynamicLap',
    ])

INFO:root:Loading metadata for study ID cc7a57880790493cb3cbd21410ea3d1c
INFO:root:Found sim types: QuasiStaticLap
INFO:root:is_exploration: True
INFO:root:Loading data for QuasiStaticLap
INFO:root:Loading metadata for study ID 178e2c8962c7450381bdb116729c22bd
INFO:root:Found sim types: DynamicLap
INFO:root:is_exploration: False
INFO:root:Loading data for DynamicLap
INFO:canopy.load_study_job:Loading job index 0


,row_index,study_id,study_name,sim_type,job_index,job_name,tLapTotal,mFuelTotal
0,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,0,,84.7487,NaN
1,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,1,,84.7487,NaN
2,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,2,,84.7487,NaN
3,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,3,,84.7487,NaN
4,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,4,,84.7487,NaN
5,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,5,,84.7487,NaN
6,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,6,,84.7487,NaN
7,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,7,,84.7487,NaN
8,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,8,,84.7487,NaN
9,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,9,,84.7487,NaN


In [ ]:
# This example will be slower as vector data has to be loaded from
# each job.
await load_worksheet_job_data(
    worksheet,
    [
        NameAndUnits('tLapTotal', 's'),
        NameAndUnits('mFuelTotal', 'kg'),
    ],
    [
        NameAndUnits('vCar', 'kph'),
        NameAndUnits('hRideF', 'mm'),
    ],
    [
        'QuasiStaticLap',
        'DynamicLap',
    ])


INFO:root:Loading metadata for study ID cc7a57880790493cb3cbd21410ea3d1c
INFO:root:Found sim types: QuasiStaticLap
INFO:root:is_exploration: True
INFO:root:Loading data for QuasiStaticLap
INFO:canopy.load_study_job:Loading job index 0
INFO:canopy.load_study_job:Loading job index 1
INFO:canopy.load_study_job:Loading job index 2
INFO:canopy.load_study_job:Loading job index 3
INFO:canopy.load_study_job:Loading job index 4
INFO:canopy.load_study_job:Loading job index 5
INFO:canopy.load_study_job:Loading job index 6
INFO:canopy.load_study_job:Loading job index 7
INFO:canopy.load_study_job:Loading job index 8
INFO:canopy.load_study_job:Loading job index 9
INFO:canopy.load_study_job:Loading job index 10
INFO:root:Loading metadata for study ID 178e2c8962c7450381bdb116729c22bd
INFO:root:Found sim types: DynamicLap
INFO:root:is_exploration: False
INFO:root:Loading data for DynamicLap
INFO:canopy.load_study_job:Loading job index 0


,row_index,study_id,study_name,sim_type,job_index,job_name,vCar_Min,vCar_Mean,vCar_Max,hRideF_Min,hRideF_Mean,hRideF_Max,tLapTotal,mFuelTotal
0,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,0,Factorial 1,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
1,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,1,Factorial 2,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
2,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,2,Factorial 3,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
3,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,3,Factorial 4,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
4,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,4,Factorial 5,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
5,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,5,Factorial 6,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
6,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,6,Factorial 7,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
7,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,7,Factorial 8,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
8,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,8,Factorial 9,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
9,0,cc7a57880790493cb3cbd21410ea3d1c,Canopy F1 hRideF Constraint,QuasiStaticLap,9,Factorial 10,73.941054,217.658312,302.430413,-14.384037,5.663982,22.595723,84.7487,NaN
